Install and import all the things

In [ ]:
!pip install serpapi

In [ ]:
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup
import requests
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

Mount and read the list of gifts for UN

In [ ]:
drive.mount('/content/drive/')
dataset = '/content/drive/MyDrive/List_gifts_for_UN.csv'
df = pd.read_csv(dataset)

# Define function

Function to find the row of the inputted gift in dataset

In [ ]:
gift_name = input("Enter the name of your gift : ")

def find_gift(gift_name, df):
      row_number = None
      for index, row in df.iterrows():
          if row['Name'].lower() == gift_name.lower():
              row_number = index
              break
      return row_number

Function to measure the significance based on the motives behind the gift

In [ ]:
def reason_significance(gift_name, df):
    row_number = find_gift(gift_name, df)
    if row_number is not None:
      row = df.iloc[row_number]
      reason_column = row['reason/ circumstance of gift'].lower()
      keywords = ["anniversary", "admission", "allegiance", "accepted"]
      significance_reasons = 0
      for keyword in keywords:
        if keyword in reason_column:
          significance_reasons = 1
          break
      gift_year = row['date of gift (either formally given/arrived in new home)']
      return significance_reasons, gift_year
    else:
      return 0, None

Function to search for the main question to know if there are relevant links

In [ ]:
import serpapi
api_key = 'YOUR API KEY'

def find_impact(gift_name, api_key):
    params = {
        "q": f"gifting the {gift_name} to UN diplomatic constraints",
        "hl": "en",
        "google_domain": "google.com",
        "api_key": api_key
    }

    search = serpapi.search(params)

    relevant_links = []
    if "organic_results" in search:
        for i in search["organic_results"]:
            relevant_links.append(i["link"])

    return relevant_links

Function to calculate the significance by adding the previous factors

In [ ]:
def calculate_significance(gift_name, df, api_key):
    significance_reasons, gift_year = reason_significance(gift_name, df)
    relevant_links = find_impact(gift_name, api_key)
    significance_impact = 1 if relevant_links else 0
    overall_significance = significance_reasons + significance_impact
    return overall_significance, significance_reasons, significance_impact, relevant_links, gift_year

Function to generate directly the answer for the main question

In [ ]:
def generate_context(gift_name, gift_year):
    prompt = f"Why is giving {gift_name} gift in {gift_year} to UN significant in the context of global history?"
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=300, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

Function to calculate the accuracy based on whether the factor which we use for defining the impact (diplomatic constraint) is found in the answer generated by the previous function

In [ ]:
def accuracy_calculation (gift_name, generated_context):
    if f"gfting the {gift_name} had an impact" or f"gifting the {gift_name} resulted in diplomatic constraints" in generated_context:
        return "The context is highly relevant and informative."
    else:
        return "The Context is somewhat general, lacking specific historical insights."

Main function

In [ ]:
def main():
    api_key = "YOUR API KEY"

    if gift_name.lower() in [name.lower() for name in df['Name'].values]:
        overall_significance, significance_reasons, significance_impact, relevant_links, gift_year = calculate_significance(gift_name, df, api_key)
        if overall_significance == 2:
            print(f"{gift_name} has a great significance")
        elif overall_significance == 1:
            print(f"{gift_name} has a good significance")
        else:
            print(f"{gift_name} has no significance")

        print(f"Overall Significance: {overall_significance}")
        print(f"Reason Significance: {significance_reasons}")
        print(f"Impact Score: {significance_impact}")

        if relevant_links:
            req = requests.get(relevant_links[0], headers={'User-Agent': 'Mozilla/5.0'})
            webpage = req.text
            html = BeautifulSoup(webpage, 'html.parser')
            paragraphs = html.select("p")
            for para in paragraphs:
                print(para.text)

        if gift_year:
            context = generate_context(gift_name, gift_year)
            print(f"\nMain Question: {context}")
            accuracy = accuracy_calculation(gift_name, context)
            print(f"\nEvaluation of the answer: {accuracy}")

    else:
        print("Gift is not found in the list.")

In [ ]:
main()